## Import Necessary Libraries

In [1]:
# External package need to install
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=8877d86d42435ab73f6a388d6813945e84935cb501c25c4ba00470356f8394ed
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
movies = pd.read_csv(
    '../input/movielens-1m-dataset/movies.csv',
    usecols = ['movieId', 'title'],
    dtype = {'movieId': int, 'title': str},
    sep = ';'
)
ratings = pd.read_csv(
    '../input/movielens-1m-dataset/ratings.csv',
    usecols = ['userId', 'movieId', 'rating'],
    dtype = {'userId': int, 'movieId': int, 'rating': float},
    sep = ';'
)

## Exploratory Data Analysis

In [4]:
movies.isnull().sum()

movieId    0
title      0
dtype: int64

In [5]:
movies.isna().sum()

movieId    0
title      0
dtype: int64

In [6]:
# Merging the movieID and the ratings DataFrames on the movieID feature
result = pd.merge(
    movies[['movieId', 'title']],
    ratings,
    on = 'movieId'
)

In [7]:
result.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,5.0
1,1,Toy Story (1995),6,4.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),9,5.0
4,1,Toy Story (1995),10,5.0


In [8]:
# Dividing the movies data set into favourtie and non favourite movies
favourite = result[result['rating'] > 3.5]
non_favourite = result[result['rating'] <= 3.5]

In [9]:
favourite.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,5.0
1,1,Toy Story (1995),6,4.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),9,5.0
4,1,Toy Story (1995),10,5.0


In [10]:
non_favourite = non_favourite[['userId', 'movieId', 'title', 'rating']]

In [11]:
non_favourite.head()

,userId,movieId,title,rating
7,21,1,Toy Story (1995),3.0
9,26,1,Toy Story (1995),3.0
10,28,1,Toy Story (1995),3.0
22,68,1,Toy Story (1995),3.0
23,73,1,Toy Story (1995),3.0


In [12]:
Non_Favourite = non_favourite.to_csv('non_favourite.csv', index=False)

In [13]:
len(favourite)

575281

In [14]:
favourite = favourite[:70000]

In [15]:
favourite.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,5.0
1,1,Toy Story (1995),6,4.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),9,5.0
4,1,Toy Story (1995),10,5.0


In [16]:
# Preparing the "basket" of movies for the association rule
basket = pd.pivot_table(
    data = favourite,
    index = 'userId',
    columns = 'title',
    values = 'rating',
    fill_value = 0
)

In [17]:
# Function to convert the feature of rated movies to binary values
def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0

In [18]:
basket_sets = basket.applymap(convert_into_binary)

In [19]:
basket_sets.head()

title,8 Seconds (1994),Above the Rim (1994),Ace Ventura,Across the Sea of Time (1995),Addams Family Values (1993),"Addiction, The (1995)","Adventures of Priscilla, Queen of the Desert, The (1994)","Age of Innocence, The (1993)","Air Up There, The (1994)",Airheads (1994),...,White Squall (1996),Widows' Peak (1994),Wild Bill (1995),Wings of Courage (1995),With Honors (1994),Wolf (1994),"Wonderful, Horrible Life of Leni Riefenstahl, The (Die Macht der Bilder) (1993)",Wyatt Earp (1994),You So Crazy (1994),"Young Poisoner's Handbook, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from apyori import apriori
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [21]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [22]:
# It will generate frequent itemsets using two-step approch
frequent_itemsets

,support,itemsets
0,0.123431,"(American President, The (1995))"
1,0.174889,(Apollo 13 (1995))
2,0.218214,(Babe (1995))
3,0.349602,(Braveheart (1995))
4,0.077277,(Casino (1995))
...,...,...
271,0.074094,"(Fugitive, The (1993), Shawshank Redemption, T..."
272,0.073386,"(Fugitive, The (1993), Shawshank Redemption, T..."
273,0.071972,"(Pulp Fiction (1994), Fugitive, The (1993), Us..."
274,0.070911,"(Fugitive, The (1993), Shawshank Redemption, T..."


In [23]:
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [24]:
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Forrest Gump (1994)),"(American President, The (1995))",0.294960,0.123431,0.074447,0.252398,2.044858,0.038040,1.172508
1,"(American President, The (1995))",(Forrest Gump (1994)),0.123431,0.294960,0.074447,0.603152,2.044858,0.038040,1.776598
2,"(Shawshank Redemption, The (1994))","(American President, The (1995))",0.361804,0.123431,0.071264,0.196970,1.595793,0.026607,1.091577
3,"(American President, The (1995))","(Shawshank Redemption, The (1994))",0.123431,0.361804,0.071264,0.577364,1.595793,0.026607,1.510037
4,(Babe (1995)),(Apollo 13 (1995)),0.218214,0.174889,0.072856,0.333874,1.909055,0.034693,1.238670


In [25]:
# As based business use case we can sort based on confidance and lift.
fav_movies = rules_mlxtend[ (rules_mlxtend['lift'] >= 1.5) & (rules_mlxtend['confidence'] >= 0.5) ]

In [26]:
fav_movies = pd.DataFrame(fav_movies)

In [27]:
fav_movies.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,"(American President, The (1995))",(Forrest Gump (1994)),0.123431,0.294960,0.074447,0.603152,2.044858,0.038040,1.776598
3,"(American President, The (1995))","(Shawshank Redemption, The (1994))",0.123431,0.361804,0.071264,0.577364,1.595793,0.026607,1.510037
7,(Apollo 13 (1995)),(Braveheart (1995)),0.174889,0.349602,0.101326,0.579373,1.657236,0.040185,1.546258
9,(Apollo 13 (1995)),(Forrest Gump (1994)),0.174889,0.294960,0.096552,0.552073,1.871686,0.044966,1.574005
11,(Apollo 13 (1995)),"(Fugitive, The (1993))",0.174889,0.285588,0.092485,0.528817,1.851678,0.042538,1.516209


In [28]:
len(fav_movies) 

329

### Let's fetch the corresponding association rules generated via Apriori algorithm for same movie and sort them in descending order of lift: ###

In [29]:
all_antecedents = [list(x) for x in fav_movies.antecedents.values]
len(all_antecedents)

329

In [30]:
all_antecedents = [list(x) for x in fav_movies.antecedents.values]
desired_indices = []
for i in range (len(all_antecedents)) : #liste de liste
    for j in range (len(all_antecedents[i])) : 
        if  all_antecedents[i][j] == "Pulp Fiction (1994)": 
            desired_indices.append(i)

apriori_recommendations = fav_movies.iloc[desired_indices,].sort_values(by=['lift'], ascending=False)
apriori_recommendations.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
926,"(Fugitive, The (1993), Pulp Fiction (1994))","(Usual Suspects, The (1995), Star Wars)",0.135809,0.171176,0.071972,0.529948,3.095925,0.048724,1.763260
914,"(Fugitive, The (1993), Pulp Fiction (1994))","(Shawshank Redemption, The (1994), Usual Suspe...",0.135809,0.178603,0.073386,0.540365,3.025507,0.049130,1.787062
804,"(Fugitive, The (1993), Pulp Fiction (1994))","(Braveheart (1995), Shawshank Redemption, The ...",0.135809,0.180548,0.071972,0.529948,2.935216,0.047452,1.743321
858,"(Braveheart (1995), Pulp Fiction (1994))","(Shawshank Redemption, The (1994), Usual Suspe...",0.152431,0.178603,0.077454,0.508121,2.844973,0.050229,1.669916
640,"(Seven (Se7en) (1995), Pulp Fiction (1994))","(Usual Suspects, The (1995))",0.110875,0.284350,0.088240,0.795853,2.798850,0.056713,3.505566


In [31]:
# TOP 5
apriori_recommendations_list = [list(x) for x in apriori_recommendations['consequents'].values]
print("Apriori Recommendations for basket containing : Pulp fiction\n")
for i in range(5):
    print("{0}: {1} with confidence of {2}"
          .format(i+1, apriori_recommendations_list[i], round(apriori_recommendations.iloc[i,5], 2)))

Apriori Recommendations for basket containing : Pulp fiction

1: ['Usual Suspects, The (1995)', 'Star Wars'] with confidence of 0.53
2: ['Shawshank Redemption, The (1994)', 'Usual Suspects, The (1995)'] with confidence of 0.54
3: ['Braveheart (1995)', 'Shawshank Redemption, The (1994)'] with confidence of 0.53
4: ['Shawshank Redemption, The (1994)', 'Usual Suspects, The (1995)'] with confidence of 0.51
5: ['Usual Suspects, The (1995)'] with confidence of 0.8


### **Code for 1 movie not basket** ###

In [32]:
s = 0
for x in basket_sets.iloc[200]:
    if  x == 1 :
        s += 1

In [33]:
basket_sets.iloc[175:190]

title,8 Seconds (1994),Above the Rim (1994),Ace Ventura,Across the Sea of Time (1995),Addams Family Values (1993),"Addiction, The (1995)","Adventures of Priscilla, Queen of the Desert, The (1994)","Age of Innocence, The (1993)","Air Up There, The (1994)",Airheads (1994),...,White Squall (1996),Widows' Peak (1994),Wild Bill (1995),Wings of Courage (1995),With Honors (1994),Wolf (1994),"Wonderful, Horrible Life of Leni Riefenstahl, The (Die Macht der Bilder) (1993)",Wyatt Earp (1994),You So Crazy (1994),"Young Poisoner's Handbook, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
191,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
199,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [34]:
fav_movies.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,"(American President, The (1995))",(Forrest Gump (1994)),0.123431,0.294960,0.074447,0.603152,2.044858,0.038040,1.776598
3,"(American President, The (1995))","(Shawshank Redemption, The (1994))",0.123431,0.361804,0.071264,0.577364,1.595793,0.026607,1.510037
7,(Apollo 13 (1995)),(Braveheart (1995)),0.174889,0.349602,0.101326,0.579373,1.657236,0.040185,1.546258
9,(Apollo 13 (1995)),(Forrest Gump (1994)),0.174889,0.294960,0.096552,0.552073,1.871686,0.044966,1.574005
11,(Apollo 13 (1995)),"(Fugitive, The (1993))",0.174889,0.285588,0.092485,0.528817,1.851678,0.042538,1.516209


In [35]:
movies_list = basket_sets.columns.tolist()
movies_list[1:5]

['Above the Rim (1994)',
 'Ace Ventura',
 'Across the Sea of Time (1995)',
 'Addams Family Values (1993)']

In [36]:
def recommendations(movie_list,movie_name) : 
    all_antecedents = [list(x) for x in movie_list.antecedents.values] #fav_movies
    desired_indices = []
    for i in range (len(all_antecedents)) : #liste de liste
        for j in range (len(all_antecedents[i])) : 
            if  all_antecedents[i][j] == movie_name : 
                desired_indices.append(i)
            
    apriori_recommendations=movie_list.iloc[desired_indices,].sort_values(by=['confidence'],ascending=False)
    return apriori_recommendations.consequents

# **Collaborative Filtering**

In [37]:
from scipy.sparse import csr_matrix

In [38]:
df_ratings = pd.read_csv(
    './non_favourite.csv',
    usecols = ['userId', 'movieId', 'rating'],
    dtype = {'userId': int, 'movieId': int, 'rating': float},
    sep=','
)

df_movie_features = df_ratings.pivot(
    index = 'movieId',
    columns = 'userId',
    values = 'rating'
).fillna(0)

mat_movie_features = csr_matrix(df_movie_features.values)

In [39]:
from sklearn.neighbors import NearestNeighbors

# apply the kNN algorithm
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# fit the dataset
model_knn.fit(df_movie_features)

movies_list = list(movies.iloc[:,1].values)
movies_to_idx = { i : movies_list[i] for i in range(0, len(movies_list)) }

In [40]:
from fuzzywuzzy import process

# create a fuzzy matcher
def fuzzy_matching(mapper, fav_movie):
    
  matched_results = process.extract(fav_movie, list(mapper.values()), limit=3)
   #print('Matching movies found:')
    
  max_ratio = 0
  max_ratio_idx = 0
  for movie in matched_results:
    #print(movie[0])
    if (movie[1] > max_ratio):
      max_ratio = movie[1]
      max_ratio_idx = list(mapper.keys())[list(mapper.values()).index(movie[0])]
        
  return max_ratio_idx

In [41]:
def get_recommendations(model_knn, data, mapper, fav_movie, n_recommendations):

    model_knn.fit(data)
    print('Input movie:', fav_movie)
    print('......\n')
    idx = fuzzy_matching(mapper, fav_movie)
    
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)

    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    reverse_mapper = {k: v for k, v in mapper.items()}

    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print(i+1, ':', reverse_mapper[idx], 'with a distance of', round(dist, 2))

In [42]:
el_aflem2 = get_recommendations(
    model_knn = model_knn,
    data = mat_movie_features,
    mapper = movies_to_idx,
    fav_movie = 'aliens',
    n_recommendations = 5
)

Input movie: aliens
......

......

Recommendations for aliens:
1 : Die Hard (1988) with a distance of 0.76
2 : Hercules (1997) with a distance of 0.76
3 : Death Wish II (1982) with a distance of 0.76
4 : Police Academy 3 with a distance of 0.73
5 : Hedd Wyn (1992) with a distance of 0.71


In [43]:
el_aflem2 = pd.DataFrame(el_aflem2)

# **Flask GUI**

In [44]:
!pip install flask
!pip install flask-ngrok

In [45]:
import pickle

In [46]:
non_fav_movie_list = pd.read_csv('non_favourite.csv')
non_fav_movie_list = pd.DataFrame(non_fav_movie_list)

In [47]:
non_fav_movie_list = non_fav_movie_list.title.unique()

In [48]:
non_fav_movie_list

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Tigerland (2000)', 'Two Family House (2000)',
       'Contender, The (2000)'], dtype=object)

In [49]:
def recommendations(movie_name) :
    set = ()
    all_antecedents = [list(x) for x in fav_movies.antecedents.values] #fav_movies
    desired_indices = []
    for i in range (len(all_antecedents)) : #liste de liste
        for j in range (len(all_antecedents[i])) : 
            if  all_antecedents[i][j] == movie_name : 
                desired_indices.append(i)
            
    apriori_recommendations=fav_movies.iloc[desired_indices,].sort_values(by=['confidence'],ascending=False)
    a = apriori_recommendations.consequents
    
    a = pd.DataFrame(a)
    
    return a

In [50]:
#CF _ modifié pour Flask
def get_recommendations(fav_movie):

    model_knn.fit(mat_movie_features)
    #print('Input movie:', fav_movie)
    #print('......\n')
    idx = fuzzy_matching(movies_to_idx, fav_movie)
    
    #print('......\n')
    distances, indices = model_knn.kneighbors(mat_movie_features[idx], n_neighbors=5+1)

    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    reverse_mapper = {k: v for k, v in movies_to_idx.items()}
    # print recommendations
    #print('Recommendations for {}:'.format(fav_movie))
    lista = []
    for i, (idx, dist) in enumerate(raw_recommends):
        lista.append(reverse_mapper[idx])
    return lista

In [51]:
get_recommendations('aliens')

['Die Hard (1988)',
 'Hercules (1997)',
 'Death Wish II (1982)',
 'Police Academy 3',
 'Hedd Wyn (1992)']

In [52]:
pickle.dump(recommendations, open('apriori_model.pkl', 'wb'))
pickle.dump(get_recommendations, open('CF_model.pkl', 'wb'))

# app.py

In [53]:
model = pickle.load(open('./apriori_model.pkl', 'rb'))
CF_model = pickle.load(open('./CF_model.pkl', 'rb'))

In [54]:
# Apriori only
from flask import Flask, render_template, request, redirect, url_for, session
from flask_ngrok import run_with_ngrok

app = Flask(__name__, static_url_path='', static_folder='../input/static', template_folder='../input/templates-2')
run_with_ngrok(app) 
app.secret_key = "pick_for_me"

@app.route('/')
@app.route('/action_to_perform_after_submission', methods=['POST', 'GET'])
def form():
    if request.method == 'POST':
        a = request.form.get('inputMovie')
        data = model(a)
        set = data.values.tolist() #fronzenset
        output1 = [list(x[0]) for x in set]
        output = []
        for x in output1:
            output.append(x[0])
        
        session['movie'] = a
        return redirect(url_for('results',output=output))
    return render_template('form.html')

@app.route('/results/<output>')
def results(output):
    a = len(output)
    output = output[2:a-2:1]
    lista = list(output.split("', '"))
    s = set()
    for x in lista: 
        s.add(x)
        
    movie = session.pop('movie', None)
    return render_template('data.html', output=s, movie=movie)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://762bb4869bd9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [55]:
# CF only
from flask import Flask, render_template, request, redirect, url_for, session
from flask_ngrok import run_with_ngrok

app = Flask(__name__, static_url_path='', static_folder='../input/static', template_folder='../input/templates-2')
run_with_ngrok(app) 
app.secret_key = "pick_for_me"

@app.route('/')
@app.route('/action_to_perform_after_submission', methods=['POST', 'GET'])
def form():
    if request.method == 'POST':
        b = request.form.get('inputMovie')
        CF_data = CF_model(b)
        output = CF_data
        session['movie'] = b
        return redirect(url_for('results', output=output))
    return render_template('form.html')

@app.route('/results/<output>')
def results(output):
    a = len(output)
    output = output[2:a-2:1]
    lista = list(output.split("', '"))
    s = set()
    for x in lista : 
        s.add(x)
    
    movie = session.pop('movie', None)
    return render_template('data.html', output=s, movie=movie)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://90471da565e5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
# Hybrid approach
from flask import Flask, render_template, request, redirect, url_for, session
from flask_ngrok import run_with_ngrok

app = Flask(__name__, static_url_path='', static_folder='../input/static', template_folder='../input/templates-2')
run_with_ngrok(app) 
app.secret_key = "pick_for_me"

@app.route('/')
@app.route('/action_to_perform_after_submission', methods=['POST', 'GET'])
def form():
    if request.method == 'POST':
        a = request.form.get('inputMovie')
        data = model(a)
        CF_data = CF_model(a)
        set = data.values.tolist() #fronzenset
        output1 = [list(x[0]) for x in set]
        output = []
        for x in output1 :
            output.append(x[0])
        
        output2 = CF_data
        session['movie'] = a
        return redirect(url_for('results',output=output,output2 = output2))
    return render_template('form.html')

@app.route('/results/<output><output2>')
def results(output,output2):
    a = len(output)
    output = output[2:a-2:1]
    lista = list(output.split("', '"))
    s = set()
    for x in lista : 
        s.add(x)
    
    movie = session.pop('movie', None)
    return render_template('data.html', output=s, output2=output2, movie=movie)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://98fad89445fe.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
